In [266]:
%matplotlib inline

import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from collections import deque
from matrixprofile import *
from scipy.stats import zscore

In [267]:
#dequeを更新
def update_deq(deq, val, ind):
    # 先端を削除するべきか判定・処理
    if deq[0][1] == ind - deq.maxlen:
        deq.popleft()
    # 要素を追加
    while(len(deq) != 0):
        if val <= deq[len(deq)-1][0]:
            deq.pop()
        else: 
            break
    deq.append((val,ind))
#     add_deq(deq, val, ind)

#dequeに要素を追加
def add_deq(deq, val, ind):
    while(len(deq) != 0):
        if val <= deq[len(deq)-1][0]:
            deq.pop()
        else: 
            break
    deq.append((val,ind))

#distance profileの集合からMPdistベクトルを計算
def calcMPdistvec_by_DistanceProfile(DPs):
    
    row_num, col_num = DPs.shape 
    MatrixProfile_BA = np.min(np.asarray(DPs), axis=0)
    MPdistvec = np.zeros(col_num - row_num + 1)
    deques = list()
    for row in range(row_num):
        deques.append(deque(maxlen=row_num))
        for col in range(row_num-1):
            add_deq(deques[row], DPs[row][col], col)
    
    start = time.time()
    temp_profile_AB = np.zeros(row_num, dtype='float64')
    for col in range(col_num - row_num + 1):        
        for row in range(row_num):

            update_deq(deques[row], DPs[row][col + (row_num-1)], col + (row_num-1))
            temp_profile_AB[row] = deques[row][0][0]
        #ソートしてthrでぶちこむ
        MPdistvec[col] = np.sort(np.concatenate([temp_profile_AB, MatrixProfile_BA[col:col + row_num]]))[0]
    print("slidemin_time = ", time.time()-start)
    return MPdistvec

# MPdistvecを計算
# query:クエリ時系列
# ts:時系列
# m:MPdictのパラメータ(部分時系列長)
def calcMPdistvec(query, ts, m):
    disP = []
    for i in range(len(query)-m+1):
        disP.append(matrixProfile.distanceProfile.massDistanceProfile(query,i,m,ts)[0])
    disP = np.asarray(disP)
    return calcMPdistvec_by_DistanceProfile(disP)

In [ ]:
tm = list()
for i in range(5000,100001,5000):
    print("time series length = ", i)
    start = time.time()
    MPdistvect = calcMPdistvec(a[500:700],a[:i],60)
    tm.append(time.time()-start)
    print("total time = ", tm[len(tm)-1]) 
    print()

time series length =  5000
slidemin_time =  0.7749946117401123
total time =  0.9413080215454102

time series length =  10000
slidemin_time =  1.5749013423919678
total time =  1.7860445976257324

time series length =  15000
slidemin_time =  2.3691084384918213
total time =  2.5103397369384766

time series length =  20000
slidemin_time =  3.2174062728881836
total time =  3.385806083679199

time series length =  25000
slidemin_time =  4.0142822265625
total time =  4.206480026245117

time series length =  30000
slidemin_time =  4.803378343582153
total time =  5.041051387786865

time series length =  35000
slidemin_time =  5.6142494678497314
total time =  5.875856637954712

time series length =  40000
slidemin_time =  6.397276878356934
total time =  6.6861395835876465

time series length =  45000
slidemin_time =  7.213687896728516
total time =  7.5529398918151855

time series length =  50000
slidemin_time =  8.065760850906372
total time =  8.504289865493774

time series length =  55000
slide

In [ ]:
plt.plot(np.linspace(5000,100000,20),tm)

Matrix Profileが最小値を取るペアを発見する

In [ ]:
def find_minimum_pair(MatP, MatPind):
    minpair = np.unique(np.union1d(np.where(MatP == MatP.min()), MatPind[np.where(MatP == MatP.min())]))
    minpair = minpair.astype('int')
    return minpair

## 人工データ

In [ ]:
trace = np.asarray(pd.read_csv('../Datasets/UCR_TS_Archive_2015/Trace/Trace_TRAIN'))
t = np.concatenate((trace[2][100:200],trace[12][100:300]))
plt.plot(t)
plt.show()
mpt, mptindex = matrixProfile.stomp(t,30)
plt.plot(mpt)
plt.show()

In [ ]:
si = np.sin(np.linspace(0,4*np.pi,60))
plt.plot(si)

In [ ]:
plt.plot(t)
plt.show()

In [ ]:
m = 30 #distance profileのクエリの長さ
disP = []
for i in range(len(si)-m+1):
    disP.append(matrixProfile.distanceProfile.massDistanceProfile(si,i,m,t)[0])
disP = np.asarray(disP)


In [ ]:
plt.plot(si)
plt.plot(t)
plt.plot(calcMPdistvec_by_DistanceProfile(disP))

In [ ]:
a = np.concatenate([disP[:,:31],disP[:,:31].T], axis=0)

In [ ]:
plt.plot(a.min(axis=1).T)
plt.show()

In [ ]:
MPdistvect = calcMPdistvec_by_DistanceProfile(disP)

In [ ]:
MPdistvect = calcMPdistvec_by_DistanceProfile(disP)

for st in range(MPdistvect.shape[0]):
    di1 = disP[:,st:st + 31].min(axis=0)
    di2 = disP[:,st:st + 31].min(axis=1)
#     plt.plot(di1)
#     plt.plot(di2)
    print(min(di1.min(),di2.min()) - MPdistvect[st])

## 線虫

In [ ]:
m = 60 # MatrixProfileの部分時系列の長さ
topN = 100000 # 線虫の頭何個使うか

データ呼び出し

In [ ]:
filenames1 = glob.glob("../Datasets/worm1/*f4")
filenames1.sort()
np.random.seed(0)
a = np.fromfile(filenames1[53],'f4')[:topN] - 10e-4*np.random.randn(topN)
a_nonoise = np.fromfile(filenames1[53],'f4')[:topN]
plt.plot(a)
plt.show()

In [ ]:
%%time
MPdistvect = calcMPdistvec(a[500:700],a,60)

In [ ]:
plt.plot(MPdistvect)

In [ ]:
# a[3300:3500] += 30 * np.sin(np.linspace(0,2*np.pi,200))
# a[4500:4700] += 30 * np.sin(np.linspace(0,2*np.pi,200))
# a[5300:5500] += 30 * np.sin(np.linspace(0,2*np.pi,200))

In [ ]:
MP, MPind = matrixProfile.stomp(a,m)
MPind = MPind.astype('int')

0の数に対するAV

In [ ]:
AV_zero = np.zeros_like(MP)

for i,_ in enumerate(AV_zero):
    AV_zero[i] = np.sum(a_nonoise[i:i+m]==0)

AV_zero -= min(AV_zero)
AV_zero /= max(AV_zero)

AV_zero = 1 - AV_zero

MP_zero = MP + (1-AV_zero) * MP.max()

plt.subplot(2,1,1)
plt.plot(MP)
plt.title("original")

plt.subplot(2,1,2)
plt.plot(MP_zero)
plt.title("Annotated")

plt.tight_layout()
plt.show()

複雑さに対する　AV

In [ ]:
plt.plot(a[0:30])

In [ ]:
AV_comp = np.zeros_like(MP)

for i,_ in enumerate(AV_comp):
    AV_comp[i] = np.sqrt(np.sum(np.diff(zscore(a[i:i+m]))**2))
    
AV_comp -= min(AV_comp)
AV_comp /= max(AV_comp)

MP_comp = MP + (1-AV_comp) * MP.max()

plt.subplot(2,1,1)
plt.plot(MP)
plt.title("original")

plt.subplot(2,1,2)
plt.plot(MP_comp)
plt.title("Annotated")

plt.tight_layout()
plt.show()

合成

In [ ]:
AV_sum = AV_zero + AV_comp
AV_sum -= min(AV_sum)
AV_sum /= max(AV_sum)

MP_sum = MP + (1-AV_sum) * MP.max()

plt.subplot(2,1,1)
plt.plot(MP)
plt.title("original")

plt.subplot(2,1,2)
plt.plot(MP_sum)
plt.title("Annotated")

plt.tight_layout()
plt.show()

In [ ]:
plt.plot(a)

In [ ]:
mindex = find_minimum_pair(MP_sum,MPind)
plt.plot(zscore(a[mindex[0]:mindex[0]+m]))
plt.plot(zscore(a[mindex[1]:mindex[1]+m]))
plt.show()

In [ ]:
find_minimum_pair(MP_comp,MPind)
plt.plot(zscore(a[mindex[0]:mindex[0]+m]))
plt.plot(zscore(a[mindex[1]:mindex[1]+m]))
plt.show()

In [ ]:
find_minimum_pair(MP,MPind)
plt.plot(zscore(a[mindex[0]:mindex[0]+m]))
plt.plot(zscore(a[mindex[1]:mindex[1]+m]))
plt.show()

In [ ]:
plt.plot(MP)
plt.show()

In [ ]:
mindex = find_minimum_pair(MP_sum,MPind)
plt.plot(zscore(a[mindex[0]:mindex[0]+m]))
plt.plot(zscore(a[mindex[1]:mindex[1]+m]))
plt.show()

In [ ]:
plt.plot((a[mindex[0]-50:mindex[0]+m+50]))
# plt.plot((a[mindex[1]:mindex[1]+m]))

In [ ]:
mindex = find_minimum_pair(MP_zero, MPind)
plt.plot(zscore(a[mindex[0]:mindex[0]+m]))
plt.plot(zscore(a[mindex[1]:mindex[1]+m]))
plt.show()

In [ ]:
a[mindex[0]:mindex[0]+m]

In [ ]:
focus = np.array([502,795,2729,4753])
focus_pair = MPind[focus]
for i, (f, fp) in enumerate(zip(focus,focus_pair)):
    plt.subplot(2,2,i+1)
    plt.plot(zscore(a[f:f+m]))
    plt.plot(zscore(a[fp:fp+m]))
plt.show()


print('f  = ', focus)
print('fp = ', focus_pair)
print('dist = ', MP[focus])

In [ ]:
mindex = find_minimum_pair(MP,MPind)

In [ ]:
mindex

In [ ]:
np.linalg.norm(zscore(a[mindex[0]:mindex[0]+m]) - zscore(a[mindex[1]:mindex[1]+m]))

In [ ]:
plt.plot(zscore(a[mindex[0]:mindex[0]+m]))
plt.plot(zscore(a[mindex[1]:mindex[1]+m]))
plt.show()

In [ ]:
plt.plot(a[3059:3259])

In [ ]:
plt.plot(AV_comp[3059:3259])
plt.show()

In [ ]:
plt.plot(MP)
plt.show()

In [ ]:
plt.plot(a[2483:2683])